<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/Iris_color_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Iris color scraping**

各分類について、webから瞳孔色が薄いものをダウンロードした

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

orig_csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/corneAI_iris_color.csv'
dst_csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/corneAI_iris_color_number.csv'


# Load the CSV file
df = pd.read_csv(orig_csv_path)

# Adding a new column 'image_id' to the dataframe
df['image_id'] = df.groupby('classification').cumcount() + 1
df['image_id'] = df['classification'] + '_' + df['image_id'].astype(str)

reordered_columns = ['image_id', 'URL', 'classification', 'comment']
df = df[reordered_columns]

# Save the dataframe to the specified path
df.to_csv(dst_csv_path, index=False)

# Display the modified dataframe
df


In [ ]:
import requests
from PIL import Image
from io import BytesIO
import os
import shutil
from tqdm import tqdm

# 保存先のディレクトリ
save_directory = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/images'

# ディレクトリが存在しない場合は作成
if os.path.exists(save_directory):
    shutil.rmtree(save_directory)
os.makedirs(save_directory)

# 複数のユーザーエージェント
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.100.0'
    'Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 Mobile Safari/537.36'
    'Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1'
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'
    'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2486.0 Safari/537.36 Edge/13.10586'
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36 OPR/34.0.2036.50'
    'Mozilla/5.0 (iPhone; CPU iPhone OS 8_4 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H143 Safari/600.1.4'
    'Mozilla/5.0 (Linux; Android 4.4; Nexus 5 Build/KRT16M) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/30.0.0.0 Mobile Safari/537.36'
    # 他のユーザーエージェントも追加可能
]

# 失敗したダウンロードのリスト
failed_downloads = []

# DataFrame内の各行に対して画像をダウンロード
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    url = row['URL']
    image_id = row['image_id']
    try:
        # 最初は標準のrequests設定で試行
        response = requests.get(url)
        if response.status_code == 200:
            # 成功した場合の処理
            image = Image.open(BytesIO(response.content))
            save_path = os.path.join(save_directory, f'{image_id}.png')
            image.save(save_path)
            print(f"Downloaded {image_id} successfully.")
        elif response.status_code == 403:
            # 403エラーの場合、異なるユーザーエージェントで試行
            for agent in user_agents:
                headers = {'User-Agent': agent}
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    image = Image.open(BytesIO(response.content))
                    save_path = os.path.join(save_directory, f'{image_id}.png')
                    image.save(save_path)
                    print(f"Downloaded {image_id} with alternate user agent.")
                    break
            else:
                # すべてのユーザーエージェントで失敗した場合
                print(f"Failed to download {image_id} after trying alternate user agents.")
                failed_downloads.append(image_id)
        else:
            # その他のエラーコードの場合
            print(f"Failed to download {image_id}: HTTP status code {response.status_code}")
            failed_downloads.append(image_id)
    except Exception as e:
        print(f"Error downloading {image_id}: {e}")
        failed_downloads.append(image_id)

# ダウンロードに失敗したimage_idのリストを表示
if failed_downloads:
    print("Failed to download the following image_ids:")
    print(failed_downloads)
else:
    print("All images downloaded successfully.")


ここで重複する画像を削除し、また、スクレイピングできなかった画像を手動でダウンロードし、images_tempフォルダにまとめる

In [101]:
orig_img_dir =  '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/images_temp'
dst_img_dir =  '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/images_submit'
if os.path.exists(dst_img_dir):
    shutil.rmtree(dst_img_dir)
os.mkdir(dst_img_dir)

orig_csv_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/corneAI_iris_color_number.csv'
dst_csv_path ='/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/corneAI_iris_color_submit.csv'

In [102]:
#######################
# 画像のないcsvの行を削除する #
#######################

import os

# List all files in the directory
file_paths = [os.path.splitext(os.path.basename(file))[0] for file in os.listdir(orig_img_dir)
              if os.path.isfile(os.path.join(orig_img_dir, file))]

# Load the CSV file
df = pd.read_csv(orig_csv_path)

# Filtering the DataFrame
df_temp = df[df['image_id'].isin(file_paths)]
df_temp

df_temp.to_csv(dst_csv_path, index=False)
df_temp


,image_id,URL,classification,comment
2,APAC_3,https://www.researchgate.net/publication/28304...,APAC,NaN
4,APAC_5,https://journals.sagepub.com/cms/10.4137/OED.S...,APAC,NaN
5,APAC_6,https://journals.sagepub.com/cms/10.4137/OED.S...,APAC,NaN
6,APAC_7,https://journals.sagepub.com/cms/10.4137/OED.S...,APAC,NaN
7,APAC_8,https://1.bp.blogspot.com/-o3nKXU9Oss8/X_Xg6XY...,APAC,NaN
...,...,...,...,...
230,tumor_21,https://www.cartereyecenter.com/wp-content/upl...,tumor,pterygium
231,tumor_22,https://www.oftalvist.es/public/especialidades...,tumor,pterygium
232,tumor_23,https://t1.uc.ltmcdn.com/es/posts/9/2/3/como_e...,tumor,pterygium
233,tumor_24,https://cdn0.remedios-naturales.org/es/posts/2...,tumor,pinguécula


In [105]:
# df_tempをDataFrameのコピーとして作成
df_temp = df_temp.copy()  # dfのスライスをコピーする

# それぞれのクラスに対して新しい番号を振る
df_temp['new_image_id'] = df_temp.groupby('classification').cumcount() + 1
df_temp['new_image_id'] = df_temp['classification'] + '_' + df_temp['new_image_id'].astype(str)

# パスの設定（.locを使用）
df_temp.loc[:, 'orig_file_path'] = orig_img_dir +'/'+ df_temp['image_id'] + '.png'
df_temp.loc[:, 'dst_file_path'] = dst_img_dir +'/'+df_temp['new_image_id'] + '.png'

# Reset the index of the DataFrame
df_temp = df_temp.reset_index(drop=True)

# Save the DataFrame to a CSV file
df_temp[['new_image_id', 'URL', 'classification', 'comment']].to_csv(dst_csv_path, index=False)


# 変更を表示
df_temp[['image_id', 'new_image_id', 'orig_file_path', 'dst_file_path']].head()

,image_id,new_image_id,orig_file_path,dst_file_path
0,APAC_3,APAC_1,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...
1,APAC_5,APAC_2,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...
2,APAC_6,APAC_3,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...
3,APAC_7,APAC_4,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...
4,APAC_8,APAC_5,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...,/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプ...


In [99]:
# import os
# import shutil
# from tqdm import tqdm

# # Assuming df_temp is a DataFrame with the file paths
# for _, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
#     orig_file_path = row['orig_file_path']
#     new_file_path = row['dst_file_path']

#     # Check if the original file exists, then move it if it does
#     if os.path.exists(orig_file_path):
#         shutil.copy(orig_file_path, new_file_path)
#     else:
#         print(f"File not found: {orig_file_path}")

100%|██████████| 213/213 [00:03<00:00, 58.50it/s]


In [106]:
##########################
# クラスごとにフォルダ分けして保存 ##
##########################


import os
import shutil
from tqdm import tqdm
import pandas as pd

# Define your directories and paths
orig_img_dir = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/images_temp'
dst_img_dir = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/Iris_color/images_submit'

# Delete and recreate the destination directory
if os.path.exists(dst_img_dir):
    shutil.rmtree(dst_img_dir)
os.mkdir(dst_img_dir)

# Process each row in the DataFrame
for _, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    image_id = row['image_id']
    class_name = image_id.split('_')[0]  # Extract class name from image_id
    class_folder = os.path.join(dst_img_dir, class_name)

    if not os.path.exists(class_folder):
        os.mkdir(class_folder)

    orig_file_path = row['orig_file_path']
    dst_file_path = row['dst_file_path']
    file_name = os.path.basename(dst_file_path)
    new_file_path = os.path.join(class_folder, file_name)

    # Check if the original file exists, then copy it if it does
    if os.path.exists(orig_file_path):
        shutil.copy(orig_file_path, new_file_path)
    else:
        print(f"File not found: {orig_file_path}")


100%|██████████| 213/213 [00:03<00:00, 59.81it/s]
